In [1]:
import io
import itertools
import numpy as np
import sklearn.metrics
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
data_train=np.load(r'C:\Users\admin\Downloads\Primary categories - Train.npz')
data_val=np.load(r'C:\Users\admin\Downloads\Primary categories - Validation.npz')
data_test=np.load(r'C:\Users\admin\Downloads\Primary categories - Test.npz')

In [ ]:
images_train=data_train['images']
labels_train=data_train['labels']

images_val=data_val['images']
labels_val=data_val['labels']

images_test=data_test['images']
labels_test=data_test['labels']

In [ ]:
images_train=images_train/255.0
images_val=images_val/255.0
images_test=images_test/255.0

In [ ]:
Epochs=15
Batch_size=64

HP_Filter_Size=hp.HParam('filter_size',hp.Discrete([3,5,7]))
HP_Filter_Num=hp.HParam('filter_number',hp.Discrete([32,64,96,128]))

METRIC_ACCURACY='accuracy'

with tf.summary.create_file_writer(r'Logs/Model 1/hparam_tuning/').as_default():
    hp.hparams_config(
    hparams=[HP_Filter_Size,HP_Filter_Num],
    metrics=[hp.Metric(METRIC_ACCURACY,display_name='Accuracy')])

In [ ]:
def train_test_model(hparams,session_num):
    model=tf.keras.Sequential([
        tf.keras.layers.Conv2D(hparams[HP_Filter_Num],hparams[HP_Filter_Size],activation='relu',input_shape=(120,90,3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(hparams[HP_Filter_Num],3,activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(3)])
    loss_rn=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(optimizer='adam',loss=loss_rn,metrics=['accuracy'])
    log_dir='Logs\\Model 1\\fit\\'+"run-{}".format(session_num)
    
    def plot_confusion_matrix(cm, class_names):
        figure = plt.figure(figsize=(8, 8))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        labels = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, labels[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        return figure
    
    def plot_to_image(figure):
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        plt.close(figure)
        buf.seek(0)
        image = tf.image.decode_png(buf.getvalue(), channels=4)
        image = tf.expand_dims(image, 0)
        return image
    
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')

    def log_confusion_matrix(epoch, logs):
        test_pred_raw = model.predict(images_val)
        test_pred = np.argmax(test_pred_raw, axis=1)
        cm = sklearn.metrics.confusion_matrix(labels_val, test_pred)
        figure = plot_confusion_matrix(cm, class_names=['Glasses/Sunglasses','Trousers/Jeans','Shoes'])
        cm_image = plot_to_image(figure)

        with file_writer_cm.as_default():
            tf.summary.image("Confusion Matrix", cm_image, step=epoch)
            
    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)
    tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1,profile_batch=0)
    
    early_stopping=tf.keras.callbacks.EarlyStopping(monitor='val_loss',mode='auto',min_delta=0,verbose=0,patience=2,restore_best_weights=True)
    model.fit(images_train,labels_train,epochs=Epochs,batch_size=Batch_size,callbacks=[early_stopping,tensorboard_callback,cm_callback],validation_data=(images_val,labels_val),verbose=2)
    _,accuracy=model.evaluate(images_val,labels_val)
    
    model.save(r'saved_models\Model 1\Run-{}'.format(session_num))
    return accuracy

In [ ]:
def run(log_dir,hparams,session_num):
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)
        accuracy=train_test_model(hparams,session_num)
        tf.summary.scalar(METRIC_ACCURACY,accuracy,step=1)

In [ ]:
session_num=1
for filter_size in HP_Filter_Size.domain.values:
    for filter_num in HP_Filter_Num.domain.values:
        hparams={
            HP_Filter_Size:filter_size,
            HP_Filter_Num:filter_num
        }
        run_name='run-%d' % session_num
        print('---Starting trial: %s' % run_name)
        print({h.name:hparams[h] for h in hparams})
        run('Logs/Model 1/hparam_tuning/'+run_name,hparams,session_num)
        
        session_num+=1

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "Logs/Model 1/hparam_tuning"

In [ ]:
%load_ext tensorboard
%tensorboard --logdir 'Logs/Model 1/fit'